In [ ]:
#Bu program Q learning kullanarak önce makineye ödül sistemi ile xox oynamayı öğretecek.
#Bir insan makineye karşı oynayabilecek.
import numpy as np
import pickle

class Durum:

    def __init__(self, player1, player2):
        self.tahta = np.zeros((3,3))
        self.player1 = player1
        self.player2 = player2
        self.bitti = False
        self.oyunTablosu = None
        # 'X'ler == 1 ve 'O'lar == -1
        self.oyuncuNumarası = 1 #player1 yani X'ler başlasın diyelim
        self.tester = [] #Buraya her round sonu kimin kazandığını yazacağız.

    def testeEkle(self, kazanan): #Kazanan oyuncuyu (1(x), -1(o) veya 0(berabere) olacak şekilde listeye ekleyeceğiz.
        self.tester.append(kazanan)

    def testiUygula(self): #Son roundları esas alarak yüzdelik bir şekilde kazanma oranlarını verecek.
        listeUzunluk = len(self.tester)
        sonBesUzunluk = int(listeUzunluk*(5/100))
        sonBesRound = self.tester[-sonBesUzunluk:]
        roundSayisi = len(sonBesRound)
        beraberlikSayisi = sonBesRound.count(0)
        beraberlikOrani = (beraberlikSayisi/roundSayisi)
        birinciOrani = (sonBesRound.count(1)/roundSayisi)
        ikinciOrani = (sonBesRound.count(-1)/roundSayisi)
        print("*****************************************************")
        print("X'lerin kazanma oranı: ",format(birinciOrani*100))
        print("O' ların kazanma oranı: ",format(ikinciOrani*100))
        print("Beraberlik oranı: ",format(beraberlikOrani*100))
        print("*****************************************************")
        if beraberlikOrani >= (70/100) and beraberlikSayisi != 0:
            print("Makine her 100 oyunun",format(beraberlikOrani*100),"kadarında berabere kaldı")
            print("Uygulama gayet düzgün çalışıyor!")

        else:
            print("Makine her 100 oyunun",format(beraberlikOrani*100),"kadarında berabere kaldı")
            print("Bu sayı 70% barajının altında kalıyor!")
            print("Round sayısını arttırarak tekrar deneyin. Aynı hatayı almaya devam ederseniz kodunuzu kontrol edin.")

    def tahtaAl(self): #Tablonun şu anki dizilimini almak için kullanılır
        self.oyunTablosu = str(self.tahta.reshape(3*3)) # 3'e 3 olduğu için 9 luk bir arraye dönüştürdük (str halinde)
        return self.oyunTablosu

    def kazanan(self): # Sırasıyla satır, sütun, ve çapraz kontrolleri sağlayarak bir kazananı veya beraberlik durumunu bulur.

        for satir in range(3): # 3x3'lük bir matriste satırlar üzerinde dolaşarak kazanan var mı diye kontrol edeceğiz.
            if sum(self.tahta[satir,:]) == 3: # X'lere 1 dediğimiz için eğer ki bir satırımız XXX olursa toplamı 3 çıkacaktır.
                self.bitti = True # Oyunu sonlandırmak için gerekli
                return 1 # X, 1'e eşittii.
            elif sum(self.tahta[satir,:]) == -3: # O'lara -1 dediğimiz için eğer ki bir satırımız OOO olursa toplamı -3 çıkacaktır.
                self.bitti = True # Oyunu sonlandırmak için gerekli
                return -1 # O, -1'e eşitti.

        for sutun in range(3): # 3x3'lük bir matriste sütunlar üzerinde dolaşarak kazanan var mı diye kontrol edeceğiz.
            if sum(self.tahta[:,sutun]) == 3:
                self.bitti = True
                return 1
            elif sum(self.tahta[:,sutun]) == -3:
                self.bitti = True
                return -1

        #Çapraz kontrol
        caprazToplam1 = sum([self.tahta[i,i] for i in range(3)])
        caprazToplam2 = sum([self.tahta[i,2-i] for i in range(3)])

        if caprazToplam1 == 3 or caprazToplam2 == 3:
            self.bitti = True
            return 1
        elif caprazToplam1 == -3 or caprazToplam2 == -3:
            self.bitti = True
            return -1

        #Beraberlik kontrolü
        if len(self.bosKonumlar()) == 0: #Oynanabilinecek boş alan kalmamışsa beraberlik olur
            self.bitti = True
            return 0

        #Eğer ki bitmemişse
        self.bitti = False
        return None


    def bosKonumlar(self): #Bu fonksiyon boş konumları tespit eden bir liste döndürecek (boş konum yoksa boş bir liste döndürecek.)
        konumlar = [] #Boş konumları daha sonrasında dolduracağımız liste
        for satir in range(3):
            for sutun in range(3):
                if self.tahta[satir,sutun] == 0: #boş ise ise x(1) veya o(-1) yoktur
                    konumlar.append((satir,sutun))
        return konumlar


    def durumuGuncelle(self, konum):
        self.tahta[konum] = self.oyuncuNumarası #Oynanan konuma oyuncu numarasına göre X veya O atanacak.
        self.oyuncuNumarası = self.oyuncuNumarası*(-1) #Hamle yapıldıktan sonra sırayı değiştirir


    def odulVer(self): #Bu fonksiyon oyun bittiği zaman "player" classına ait feedReward() fonksiyonu ile ödül verir.
        result = self.kazanan() #kazanma veya beraberlik durumlarına göre 1, -1 veya 0 a eşit olacaktır

            #(Kazananma durumuna göre) Son hamleden ilk hamleye doğru azalacak şekilde ödül veriyoruz
            #Aslında ceza ve ödül fonksiyonları birbirinin aynısı sadece girilen değer kazanan ve kaybedene göre pozitif veya negatif diye ayarlanacak
        if result == 1:
            self.player1.feedReward(1)
            self.player2.feedCeza(-1)
        elif result == -1:
            self.player1.feedCeza(-1)
            self.player2.feedReward(1)
        elif result == 0:
            self.player1.feedReward(0.5)
            self.player2.feedReward(1)


    def reset(self): #Oyunu ve tahtayı sıfırlar
        self.tahta = np.zeros((3,3))
        self.bitti = False
        self.oyunTablosu = None
        self.oyuncuNumarası = 1


    def machineLearning(self, rounds=1000): # Makine kendi kendine oynayarak q learning metodu ile eğitilecek
        for i in range(rounds):
            if i%100 == 0:
                print("Rounds ()", format(i)) #100 maçta bir bilgilendirecek bizi
            while not self.bitti: #Round bitene kadar çalışacak
                #Oyun 1. oyuncudan başladığı için player1
                konumlar = self.bosKonumlar()
                player1Aksiyon = self.player1.aksiyonSec(konumlar, self.tahta, self.oyuncuNumarası) #Aksiyon seçiyoruz
                #Durumu az önce seçtiğimiz aksiyon ile güncelliyoruz
                self.durumuGuncelle(player1Aksiyon) #Hamleyi tahtaya ekleyip oyuncu sırasını değiştiriyoruz
                oyun_Tablosu = self.tahtaAl()
                self.player1.durumEkle(oyun_Tablosu) #Hamleyi daha sonra puanlamak için kaydediyoruz (state ekleyecek)

                win = self.kazanan()
                if win is not None: #Oyunun bitip bitmediğini kontrol ediyoruz
                    if win == 1:
                        print("Birinci oyuncu kazandı!") #Birinci oyuncu kazandıysa win == 1 olacaktır ve az önce hamlesini yaptı
                    elif win == 0:
                        print("Berabere bitti!!!") #berabere bittiyse win == 0 olacaktır.

                    self.testeEkle(win) #Sonucu daha sonrasında uygulamanın çalışıp çalışmadığını kontrol etmek için listeye ekliyoruz
                    self.odulVer() #Öğretmesi için son hamleden ilk hamleye doğru hamlelere ödül verir
                    #Round bittiği için her şeyi sıfırlıyoruz
                    self.player1.reset()
                    self.player2.reset()
                    self.reset()
                    break #Roundu bitirir



                else:

                    #Player 2'nin sırası (oyun bitmediyse)
                    konumlar = self.bosKonumlar()
                    player2Aksiyon = self.player2.aksiyonSec(konumlar, self.tahta, self.oyuncuNumarası) #Aksiyon seçiyoruz
                    #Durumu az önce seçtiğimiz aksiyon ile güncelliyoruz
                    self.durumuGuncelle(player2Aksiyon) #Hamleyi tahtaya ekleyip oyuncu sırasını değiştiriyoruz
                    oyun_Tablosu = self.tahtaAl()
                    self.player2.durumEkle(oyun_Tablosu) #Hamleyi daha sonra puanlamak için kaydediyoruz (state ekleyecek)

                    win = self.kazanan()
                    if win is not None: #Oyunun bitip bitmediğini kontrol ediyoruz
                        if win == -1:
                            print("İkinci oyuncu kazandı!") #İkinci oyuncu az önce hamlesini yaptı eğer ki kazandıysa win == -1 olacaktır
                        elif win == 0:
                            print("Berabere bitti!!!")

                        self.testeEkle(win) #Sonucu daha sonrasında uygulamanın çalışıp çalışmadığını kontrol etmek için listeye ekliyoruz
                        self.odulVer() #Öğretmesi için son hamleden ilk hamleye doğru hamlelere ödül verir
                        #Round bittiği için her şeyi sıfırlıyoruz
                        self.player1.reset()
                        self.player2.reset()
                        self.reset()
                        break #Roundu bitirir

    def play(self): #İnsana karşı oynamak için
        while not self.bitti: #Round bitene kadar çalışacak
            #Birinci oyuncu başlıyor
            konumlar = self.bosKonumlar()
            tahta = self.tahta
            oyuncuNum = self.oyuncuNumarası
            player1Aksiyon = self.player1.aksiyonSec(konumlar, tahta, oyuncuNum) #Aksiyon seçiyoruz
            self.durumuGuncelle(player1Aksiyon) #Hamleyi tahtaya ekleyip oyuncu sırasını değiştiriyoruz
            self.tahtaGoster()

            win = self.kazanan() #fonksiyon x'ler kazanırsa 1 o'lar kazanırsa -1 berabere biterse 0 döndürecektir.
            if win is not None: #Bittiyse None değeri değişecektir.
                if win == 1:
                    print(self.player1.name, "Kazandı!!!")
                elif win == 0:
                    print("Berabere bitti!!!")
                self.reset() #oyunu reset fonksiyonu ile sıfırlıyoruz.
                break

            else:
                #ikinci oyuncu devam ediyor
                konumlar = self.bosKonumlar()
                tahta = self.tahta
                oyuncuNum = self.oyuncuNumarası
                player2Aksiyon=self.player2.aksiyonSec(konumlar, tahta, oyuncuNum) #Aksiyon seçiyoruz
                self.durumuGuncelle(player2Aksiyon) #Hamleyi tahtaya ekleyip oyuncu sırasını değiştiriyoruz
                self.tahtaGoster()

                win = self.kazanan() #fonksiyon x'ler kazanırsa 1 o'lar kazanırsa -1 berabere biterse 0 döndürecektir.
                if win is not None: #Bittiyse None değeri değişecektir.
                    if win == -1:
                        print(self.player2.name, "Kazandı!!!")
                    elif win == 0:
                        print("Berabere bitti!!!")
                    self.reset() #oyunu reset fonksiyonu ile sıfırlıyoruz.
                    break

    def tahtaGoster(self):
        #Player 1 X,  Player 2 O' olarak oynuyor.
        for satir in range(3): #satırlar üzerinde gezmek için kullanacağız
            print("-------------")
            for sutun in range(3): #sutunlar uzerinde gezmek için kullanacağız
                if self.tahta[satir,sutun] == 1: #X lerin yerini bulacağız
                    print("|" + "X" + "|", end="")
                elif self.tahta[satir,sutun] == -1: #O ların yerini bulacağız
                    print("|" + "O" + "|", end="")
                elif self.tahta[satir,sutun] == 0: #boşlukların yerini bulacağız
                    print("|" + " " + "|", end="")
            print("")
        print("-------------")



In [ ]:
class Player: # xox Botu
    def __init__(self, name, learning=0.2, expRate=0.90, expRateAzaltma = 0, azaltmaKatsayisi=0.9):
        self.name = name
        self.states = [] #Burası konumları her hamleden sonra kaydedeceğimiz yer. Bu sayede Q Learning metodu ile son hamleden ilk hamleye doğru puan verebileceğiz.
        self.learning = learning
        self.expRate = expRate
        self.azaltmaKatsayisi = azaltmaKatsayisi
        self.states_value = {} #Bu boş listeye oynadığımız hamleler için anahtar kilit uyumu ile puanlar verilecek (Q learning)
        self.expRateAzaltma = expRateAzaltma #Bu sayı her round sonunda expRate den çıkacak ve ilk roundlarda rastgele sallama ihtimali çok olurken son roundlarda az olacak.

    def tahtaAl(self, tahta):
        oyunTablosu = str(tahta.reshape(3*3)) #3x3 olan tahtamızı bir boyutlu 9 elemanlı bir arraye çevirdik
        return oyunTablosu


    def aksiyonSec(self, konumlar, mevcutTahta, sembol):
        if np.random.uniform(0,1) <= self.expRate: #Burada ayarladığımız expRate değerine göre rastgele bir aksiyon alacak. Bir değer girilmezse %30 ihtimal.
            index = np.random.choice(len(konumlar)) #Oynanabilinecek konumlara göre (0 lara göre) rastgele bir konum yer seçer
            aksiyon = konumlar[index] #O yerin bulunduğu konum oynayacağı aksiyon olur
            return aksiyon


        else: #Rastgele bir hamle yapmayacaksa Q learning sayesinde edindiği bilgilerle en yüksek puanlı yeri oynayacak
            maxDeger = -100000 #Burada max değeri çok düşük bir değer seçiyoruz ki sonraki herhangi bir değer bu değerden daha büyük olsun.
            for konum in konumlar: #Boş konumlar üzerinde gezinerek en yüksek puanı arayacağız
                sonrakiTahta = mevcutTahta.copy()
                sonrakiTahta[konum] = sembol
                sonrakiOyunTablosu = self.tahtaAl(sonrakiTahta)

                if self.states_value.get(sonrakiOyunTablosu) == None: #"sonrakiOyunTablosu" değerinin sözlükte bir karşılığı yoksa value 0 olur
                    value = 0
                else:
                    value = self.states_value.get(sonrakiOyunTablosu) #"sonrakiOyunTablosu" değerinin sözlükte bir karşılığı varsa value o değer olur

                if value >= maxDeger: #yeni max değeri atayacağız
                    maxDeger = value
                    aksiyon = konum #en yüksek değerdeki konumumuz aksiyon olacak
            return aksiyon

    def durumEkle(self, state):
        self.states.append(state) #Burası konumları her hamleden sonra kaydedeceğimiz yer. Bu sayede Q Learning metodu ile son hamleden ilk hamleye doğru puan verebileceğiz.


    def feedReward(self, reward): #Ödül verme sisteminin çalışacağı yer burası olacak. Q learning formülünü yazacağız.
        for st in reversed(self.states): #En sondan başa doğru puan vermek için listeyi tersten dolaşacak ve en yüksek puanı son hamleye verecek.
            if self.states_value.get(st) == None: #eğer ki bu hamlenin sözlükte bir karşılığı (değeri) yoksa 0 atayacağız
                self.states_value[st] = 0

            self.states_value[st] +=self.learning*(self.azaltmaKatsayisi * reward - self.states_value[st]) #Bu bir Q learning formülü. Azaltma katsayısı sayesinde sondan başa doğru azalacak şekilde ödül verecek.

            reward = self.states_value[st]


    def feedCeza(self, reward): #Ödül verme sisteminin çalışacağı yer burası olacak. Q learning formülünü yazacağız.
        for st in reversed(self.states): #En sondan başa doğru puan vermek için listeyi tersten dolaşacak ve en düşük puanı son hamleye verecek.
            if self.states_value.get(st) == None: #eğer ki bu hamlenin sözlükte bir karşılığı (değeri) yoksa 0 atayacağız
                self.states_value[st] = 0

            self.states_value[st] +=self.learning*(self.azaltmaKatsayisi * reward - self.states_value[st]) #Bu bir Q learning formülü. Azaltma katsayısı sayesinde sondan başa doğru azalacak şekilde ceza verecek.

            reward = self.states_value[st] #bu sefer ceza oluyor




    def reset(self): #Kaydedilmiş hamleleri resetlemiş olacağız
        self.states = []
        if self.expRate>0.05:
            self.expRate = self.expRate - self.expRateAzaltma


    def savePolicy(self):
        print("Değerler kaydedildi")
        print(self.states_value) #Tek tek atadığımız değerleri görmek için

        fw = open('policy_' + str(self.name), 'wb')
        pickle.dump(self.states_value, fw) #pickle kütüphanesi kullanarak kaydettik.
        fw.close()


    def loadPolicy(self, filename):
        fr = open(filename, 'rb') #Kaydettiğimiz dosyayı açtık
        self.states_value = pickle.load(fr) #dosyadaki değerleri states_value kısmına aktardır
        fr.close()



In [ ]:
class HumanPlayer: #insan oyuncu
    def __init__(self, name):
        self.name = name

    def aksiyonSec(self, konumlar, tahta, oyuncuNum):
        while True:
            satirr=int(input("Satır Değeri Giriniz: "))
            sutunn=int(input("Sütun Değeri Giriniz: "))
            aksiyon = (satirr, sutunn)
            if aksiyon in konumlar: #Eğer ki oynanabiliniyorsa seçilen hamle
                return aksiyon

    def durumEkle(self, state):
        pass

    def feedReward(self, reward):
        pass

    def reset(self):
        pass



In [ ]:
round = int(input("Round sayısını giriniz: "))
azaltma = ((0.85 / round))

Oyuncu1 = Player("Oyuncu1", expRateAzaltma = azaltma)
Oyuncu2 = Player("Oyuncu2", expRateAzaltma = azaltma)

st = Durum(Oyuncu1, Oyuncu2)
print("Bot öğrenmeye başladı.")

st.machineLearning(rounds = round)
st.testiUygula()

Görüntülenen çıkış son 5000 satıra kısaltıldı.
Birinci oyuncu kazandı!
Birinci oyuncu kazandı!
Berabere bitti!!!
Berabere bitti!!!
Berabere bitti!!!
Berabere bitti!!!
Berabere bitti!!!
Birinci oyuncu kazandı!
Birinci oyuncu kazandı!
Berabere bitti!!!
Berabere bitti!!!
Berabere bitti!!!
Birinci oyuncu kazandı!
Berabere bitti!!!
Berabere bitti!!!
Berabere bitti!!!
Berabere bitti!!!
Berabere bitti!!!
İkinci oyuncu kazandı!
Birinci oyuncu kazandı!
Berabere bitti!!!
Berabere bitti!!!
Birinci oyuncu kazandı!
Berabere bitti!!!
Birinci oyuncu kazandı!
Birinci oyuncu kazandı!
Birinci oyuncu kazandı!
Berabere bitti!!!
Birinci oyuncu kazandı!
Berabere bitti!!!
Berabere bitti!!!
Birinci oyuncu kazandı!
Berabere bitti!!!
Birinci oyuncu kazandı!
İkinci oyuncu kazandı!
Birinci oyuncu kazandı!
Berabere bitti!!!
Berabere bitti!!!
Berabere bitti!!!
Birinci oyuncu kazandı!
Birinci oyuncu kazandı!
Berabere bitti!!!
Berabere bitti!!!
Birinci oyuncu kazandı!
Rounds () 25100
Berabere bitti!!!
Birinci oyuncu 

In [ ]:
Oyuncu1.savePolicy()
Oyuncu2.savePolicy()

Değerler kaydedildi
{'[-1.  1.  1.  1. -1. -1. -1.  1.  1.]': 0.4499999999999999, '[-1.  1.  0.  1. -1.  0. -1.  1.  1.]': 0.3353304574950127, '[ 0.  0.  0.  1. -1.  0. -1.  1.  1.]': -0.8061733362432625, '[ 0.  0.  0.  0. -1.  0.  0.  1.  1.]': 0.4924888246134633, '[0. 0. 0. 0. 0. 0. 0. 1. 0.]': 0.23590941996873568, '[-1. -1.  1.  0.  0.  1. -1.  1.  1.]': 0.8381524709376, '[-1. -1.  1.  0.  0.  0.  0.  1.  1.]': 0.3083609072014248, '[-1.  0.  1.  0.  0.  0.  0.  0.  1.]': 0.5700750307068448, '[0. 0. 0. 0. 0. 0. 0. 0. 1.]': 0.28990410488033885, '[ 1.  1.  1. -1.  1. -1. -1.  1. -1.]': 0.8986073174558039, '[ 1.  0.  1.  0.  1. -1. -1.  1. -1.]': 0.4485683109832521, '[ 1.  0.  1.  0.  1. -1. -1.  0.  0.]': 0.8084884210534931, '[ 1.  0.  1.  0.  0.  0. -1.  0.  0.]': 0.3720715648465338, '[1. 0. 0. 0. 0. 0. 0. 0. 0.]': 0.25002987326469145, '[ 1. -1. -1.  0. -1.  1.  1.  0.  1.]': -0.1657283496232731, '[ 1.  0. -1.  0. -1.  1.  1.  0.  0.]': 0.5411713632048364, '[ 1.  0. -1.  0.  0.  0.  1

In [ ]:
Oyuncu1.loadPolicy("policy_Oyuncu1")
Oyuncu2.loadPolicy("policy_Oyuncu2")

In [ ]:
Oyuncu1 = Player("BilgisayarX", expRate=0, expRateAzaltma=azaltma)
Oyuncu1.loadPolicy("policy_Oyuncu1")
Oyuncu2 = Player("BilgisayarO", expRate=0, expRateAzaltma=azaltma)
Oyuncu2.loadPolicy("policy_Oyuncu2")
Human = HumanPlayer("Mert")
while True:
    if str(input("Oyuna X olarak başlamak ister misin?(evet yada hayır): ")).lower() == "evet":
        st = Durum(Human, Oyuncu2)
        st.play()
        if str(input("Tekrar oynamak ister misin?(evet yada hayır): ")).lower() == "hayır":
            break
    else:
        st = Durum(Oyuncu1, Human)
        st.play()
        if str(input("Tekrar oynamak ister misin?(evet yada hayır): ")).lower() == "hayır":
            break

Oyuna X olarak başlamak ister misin?(evet yada hayır): evet
Satır Değeri Giriniz: 1
Sütun Değeri Giriniz: 1
-------------
| || || |
-------------
| ||X|| |
-------------
| || || |
-------------
-------------
| || ||O|
-------------
| ||X|| |
-------------
| || || |
-------------
Satır Değeri Giriniz: 0
Sütun Değeri Giriniz: 0
-------------
|X|| ||O|
-------------
| ||X|| |
-------------
| || || |
-------------
-------------
|X|| ||O|
-------------
| ||X|| |
-------------
| || ||O|
-------------
Satır Değeri Giriniz: 1
Sütun Değeri Giriniz: 2
-------------
|X|| ||O|
-------------
| ||X||X|
-------------
| || ||O|
-------------
-------------
|X|| ||O|
-------------
|O||X||X|
-------------
| || ||O|
-------------
Satır Değeri Giriniz: 0
Sütun Değeri Giriniz: 2
Satır Değeri Giriniz: 2
Sütun Değeri Giriniz: 0
-------------
|X|| ||O|
-------------
|O||X||X|
-------------
|X|| ||O|
-------------
-------------
|X||O||O|
-------------
|O||X||X|
-------------
|X|| ||O|
-------------
Satır Değeri